### Material definition

In [20]:
from dolfinx_materials.material import MFrontMaterial

general_properties = {
    "xyoung": 210e3,
    "xnu": 0.3,
    "R0": 507.0,
    "Rinf": 818.0,
    "b": 9.14,
    "q1": 1.5,
    "q2": 1.0,
    "q3": 2.25,
    "fc": 0.1,
    "fr": 0.25,
    "fn": 0.02,
    "en": 0.3,
    "sn": 0.1,
    "fini": 4e-3,
}

material = MFrontMaterial("src/libBehaviour.so", "GTN")
material.material_properties = {
    k: general_properties[k] for k in material.material_property_names
}
print("Material parameters:", material.material_property_names)
print("Gradients (inputs):", material.gradient_names)
print("Fluxes (outputs):", material.flux_names)
print("Internal state variables:", material.internal_state_variable_names)

Material parameters: ['xyoung', 'xnu', 'R0', 'Rinf', 'b', 'q1', 'q2', 'q3', 'fc', 'fr', 'fn', 'en', 'sn']
Gradients (inputs): ['DeformationGradient']
Fluxes (outputs): ['FirstPiolaKirchhoffStress']
Internal state variables: ['ElasticStrain', 'Porosity', 'EquivalentPlasticStrain', 'Broken']


## Problem implementation

In this large-strain setting, the `QuadratureMapping` acts from the deformation gradient $\boldsymbol{F}=\boldsymbol{I}+\nabla\boldsymbol{u}$ to the first Piola-Kirchhoff stress $\boldsymbol{P}$. We will work in a Total Lagrangian formulation, writing the weak form of equilibrium on the reference configuration $\Omega_0$, thereby defining the nonlinear residual weak form as (no applied external forces):
Find $\boldsymbol{u}\in V$ such that:
$$
\int_{\Omega_0} \boldsymbol{P}(\boldsymbol{F}(\boldsymbol{u})):\nabla \boldsymbol{v} \,\text{d}\Omega = 0 \quad \forall \boldsymbol{v}\in V
$$
Here the constitutive relation $\boldsymbol{P}(\boldsymbol{F})$ will be provided by the `QuadratureMap` loading a specific `MFront` file. Note that, in the above expression, we omitted internal state variables and their evolution equations

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import ufl
from mpi4py import MPI
from dolfinx import fem
from dolfinx.cpp.nls.petsc import NewtonSolver
from dolfinx_materials.quadrature_map import QuadratureMap
from dolfinx_materials.solvers import NonlinearMaterialProblem
from dolfinx_materials.utils import axi_grad, nonsymmetric_tensor_to_vector

from mealor import DirichletBoundaryCondition
from mealor.utils import integrate, evaluate_on_points, save_to_file
from geometry import generate_NT
from load_stepping import LoadSteppingStrategy

## Define geometry and mesh (quadrangles)
R = 4.0            # notch radius
height = 23.0      # half-height of the specimen
Phi = 18.0         # maximum diameter
Phi_0 = 10.0       # minmum diameter
coarse_size = 2.0  # coarse mesh size
fine_size = 0.4    # fine mesh size
domain, cell_markers, facet_markers = generate_NT(height, Phi, Phi_0, R, coarse_size, fine_size)


## Setup function space, quadrature degree and integration measures
V = fem.VectorFunctionSpace(domain, ("CG", 1))
deg_quad = 1        # 1 Gauss points for linear quadrangles
ds = ufl.Measure(
    "ds",
    domain=domain,
    subdomain_data=facet_markers,
    metadata={"quadrature_degree": deg_quad},
)

## Boundary conditions
Uimp = fem.Constant(domain, 1.0)
dirichlet = DirichletBoundaryCondition(V)
dirichlet.add_bc_topological(facet_markers, 1, uy=0)
dirichlet.add_bc_topological(facet_markers, 2, ux=0, uy=Uimp)
dirichlet.add_bc_topological(facet_markers, 3, ux=0)


## Test, trial and unknown function
v = ufl.TestFunction(V)
du = ufl.TrialFunction(V)
u = fem.Function(V, name="Displacement")

# Dummy function used to compute force reaction
# fill with u=1 on imposed displacement boundary and use residual
v_reac = fem.Function(V)
fem.set_bc(v_reac.vector, dirichlet.bcs)

## Quadrature map object, needs to declare the "DeformationGradient" as a UFL expression of "u"
qmap = QuadratureMap(domain, deg_quad, material)
x = ufl.SpatialCoordinate(domain)
def F(u):
    return nonsymmetric_tensor_to_vector(ufl.Identity(3) + axi_grad(x[0], u))
qmap.register_gradient("DeformationGradient", F(u))


## Define non-linear residual weak form
# Note: in axisymmetric conditions, dx = dr*dz for the 2D mesh,
# hence, we use r = x[0] to write dOmega = r*dr*dz
PK1 = qmap.fluxes["FirstPiolaKirchhoffStress"]
Res = ufl.dot(PK1, ufl.derivative(F(u), u, v)) * x[0] * qmap.dx
Jac = qmap.derivative(Res, u, du)
problem = NonlinearMaterialProblem(qmap, Res, Jac, u, dirichlet.bcs)


# Create Newton solver
newton = NewtonSolver(MPI.COMM_WORLD)
newton.rtol = 1e-6
newton.atol = 1e-6
newton.convergence_criterion = "residual"
newton.max_it = 100


## Recover and initialize internal state variables
# Initialize initial porosity with fini
f = qmap.internal_state_variables["Porosity"]
fini = general_properties["fini"]
f.vector.array[:] = fini
qmap.update_initial_state("Porosity")
# Recover broken state variable
broken = qmap.internal_state_variables["Broken"]


## Define load stepping strategy 
# (heuristic to adapt time step to target a specific porosity increment `target_df`)
target_df = 1e-3  # target porosity increase
dU_max = 1e-2  # maximum displacement increment
dU_min = 1e-4  # minimum displacement increment
load_stepper = LoadSteppingStrategy(target_df, fini, dU_min, dU_max)


## Output file names
prefix = f"{material.name}_mesh_{fine_size}"
out_file = prefix + f"/results_R{R}.xdmf"


## Load-stepping loop
U_max = 1.5  # final displacement
dU = 1e-3  # first displacement increment
U = 0  # initial total displacement
i = 0
problem_stats = []
while U < U_max:
    i += 1
    U += dU
    Uimp.value = U
    dirichlet.update()

    # Solve nonlinear problem with Newton method
    converged, it = problem.solve(newton)

    # Compute number of broken points
    bp = broken.vector.array[:]
    num_broken = sum(broken.vector.array[:])
    print("Number of broken points", num_broken)

    # Compute maximum porosity for non-broken points
    f_max = max(f.vector.array[np.logical_not(bp)])

    # Update load step
    dU = load_stepper.new_step(dU, f_max)

    # Get internal state variables as DG-0 functions for output
    porosity = qmap.project_on("Porosity", ("DG", 0)) 
    p = qmap.project_on("EquivalentPlasticStrain", ("DG", 0))
    
    # Output to files
    rewrite = (i==1)
    save_to_file(out_file, u, t=U, rewrite=rewrite)
    save_to_file(out_file, porosity, t=U, rewrite=rewrite)
    save_to_file(out_file, p, t=U, rewrite=rewrite)

    # Evaluate fields at given points
    uv = evaluate_on_points(u, [Phi_0/2, 0, 0])
    pv = evaluate_on_points(p, [0.,0.,0.])
    DeltaPhi = -2*uv[0]
    
    
    # Nominal stress calculation
    S0 = np.pi * Phi_0**2 / 4
    # PK1_0 = qmap.project_on("FirstPiolaKirchhoffStress", ("DG", 0))
    Force = 2 * np.pi*integrate(ufl.action(Res, v_reac))


    print(
        f"Increment {i}\n  Strain: {U/height:.4f}   Stress: {Force/S0:.3f} MPa   max porosity: {f_max:.5f}    strain_increment: {dU:.5f}    porosity increment: {load_stepper.df_max:.5f}"
    )

    # Save to file
    problem_stats.append([U / height, f_max, pv, Force / S0, DeltaPhi, num_broken])
    results = np.asarray(problem_stats)
    np.savetxt(
        prefix + f"/results_R{R}.csv",
        results,
        delimiter=",",
        header="Strain, f_max, p, Sigma, DeltaPhi, num_broken",
    )



Solution reached in 2 iterations.
Number of broken points 0.0
Increment 1
  Strain: 0.0000   Stress: 21.023 MPa   max porosity: 0.00400    strain_increment: 0.00200    porosity increment: 0.00000
Solution reached in 2 iterations.
Number of broken points 0.0
Increment 2
  Strain: 0.0001   Stress: 63.060 MPa   max porosity: 0.00400    strain_increment: 0.00400    porosity increment: 0.00000
Solution reached in 2 iterations.
Number of broken points 0.0
Increment 3
  Strain: 0.0003   Stress: 147.100 MPa   max porosity: 0.00400    strain_increment: 0.00800    porosity increment: 0.00000
Solution reached in 2 iterations.
Number of broken points 0.0
Increment 4
  Strain: 0.0007   Stress: 315.039 MPa   max porosity: 0.00400    strain_increment: 0.01000    porosity increment: 0.00000
Solution reached in 5 iterations.
Number of broken points 0.0
Increment 5
  Strain: 0.0011   Stress: 517.524 MPa   max porosity: 0.00401    strain_increment: 0.01000    porosity increment: 0.00001
Solution reached 

RuntimeError: Failed to create/open HDF5 file (append mode).